In [5]:
import os
from download import download
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

url = "http://vam.wfp.org/sites/data/WFPVAM_FoodPrices_05-12-2017.csv"
file_name = 'WFPVAM_FoodPrices_05-12-2017.csv'

def get_dataset(url, file_name):
    if not os.path.isfile(file_name):
        download(url, file_name)
    
    return pd.read_csv(file_name, encoding='latin-1')

df = get_dataset(url, file_name)

In [42]:
#new = df.groupby("adm1_name")
#new.describe()

dict = {}
for (product, unit), df_group in df.groupby(["cm_name", "um_name"]):
    if not product in dict:
        dict[product] = [unit]
    else:
        dict[product].append(unit)

dict2 = {key: value for key, value in dict.items() if len(value) > 1}
print(dict2)


{'Apples (red)': ['500 G', 'KG'], 'Bananas': ['500 G', 'KG', 'Pound'], 'Beans': ['100 KG', 'KG'], 'Beans (black)': ['45 KG', 'MT', 'Marmite', 'Pound'], 'Beans (dry)': ['500 G', '90 KG', 'KG'], 'Beans (niebe)': ['90 KG', 'KG'], 'Beans (red)': ['45 KG', 'KG', 'MT', 'Marmite'], 'Bread': ['150 G', '400 G', '500 G', 'KG', 'Pound', 'Unit'], 'Bread (wheat)': ['Loaf', 'Unit'], 'Cabbage': ['500 G', 'KG'], 'Carrots': ['500 G', 'KG'], 'Cashew': ['500 G', 'KG'], 'Cassava': ['3.5 KG', '500 G', '91 KG', 'KG'], 'Cassava (cossette)': ['90 KG', 'KG'], 'Cassava (dry)': ['3.5 KG', 'KG'], 'Cassava (fresh)': ['50 KG', 'KG'], 'Cheese (dry)': ['KG', 'Pound', 'Unit'], 'Cheese (picon)': ['160 G', '168 G', 'KG'], 'Cocoa': ['KG', 'Libra'], 'Coffee': ['KG', 'Libra', 'Pound'], 'Coffee (instant)': ['500 G', '85 G', 'Unit'], 'Cowpeas (brown)': ['100 KG', 'KG'], 'Eggs': ['1.8 KG', '10 pcs', '2 KG', '30 pcs', '650 G', 'Dozen', 'KG', 'Unit'], 'Fish (dry)': ['500 G', 'KG'], 'Fish (sardine, canned)': ['125 G', 'KG'], 'Fi